In [ ]:
%pip install tqdm
%pip install gensim

In [1]:
import pandas as pd

from tqdm import tqdm

from arena_util import remove_seen
from arena_util import write_json

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, remove_stopwords, stem_text, strip_multiple_whitespaces

In [2]:
# filters used for preprocess_string
CUSTOM_FILTERS = [remove_stopwords, stem_text, strip_punctuation, strip_multiple_whitespaces]

### 데이터 불러오기

In [3]:
train = pd.read_json('./arena_data/orig/train.json', encoding='UTF-8')
val = pd.read_json('./arena_data/questions/val.json', encoding='UTF-8')
song_meta = pd.read_json('./data/song_meta.json', encoding='UTF-8')
most_results = pd.read_json('./arena_data/results/results.json', encoding='UTF-8')

**preprocess_string의 결과**  
그냥 preprocess_string은 2글자인 단어들은 버림  
ex) 듣기만해도 신나는 노래(남자 아이돌) --> 듣기만, 신나는, 아이돌

In [29]:
idx = 44
print(preprocess_string(val['plylst_title'][idx]))
print(preprocess_string(val['plylst_title'][idx], CUSTOM_FILTERS))

['듣기만', '신나는', '아이돌']
['듣기만', '해도', '신나는', '노래', '남자', '아이돌']


In [4]:
min_count = 10
size = 100
window = 100
sg = 1

In [5]:
p2v_model = WordEmbeddingsKeyedVectors(size)

In [6]:
# pandas apply 진행상황 보여주기 위한 모듈
tqdm.pandas()

### Dictionary 만들기

- song_dic  
    plylst id와 songs 배열을 매칭
- tag_dic  
    plylst id와 tags 배열을 매칭
- **song_name_dic**  
    plylst id와 song name을 분해한 list를 매칭(plylst id - \[song1 분해, song2 분해, ... \])
- **total(pandas.Series)**  
    song name 분해한 것 + tag + plylst 제목 분해한 것

In [ ]:
# album name을 추가할 때 활용
def preproc_song(song_id):
    song_pre_str = preprocess_string(song_meta.loc[song_id, 'song_name'])
    try:
        song_pre_str += preprocess_string(song_meta.loc[song_id, 'album_name'])
    except:
        pass
    return song_pre_str

아래는 노래 제목만 추가하는 버전  
노래 순서를 섞어보면?

In [7]:
import random

def shuffle_list(seed, lst):
    random.seed(seed)
    random.shuffle(lst)
    return lst

def preproc_shuffle_song(seed, songs):
    songs = shuffle_list(seed, songs)
    return [preprocess_string(song_meta.loc[song_id, 'song_name'], CUSTOM_FILTERS) for song_id in songs]

In [8]:
song_dic = {} # plylst id - song id
song_name_dic = {} # plylst id - [preprocessed song name]
tag_dic = {} # plylst id - tag
data = pd.concat([train, val])
data = data.set_index('id')

song_dic = data['songs'].to_dict()
tag_dic = data['tags'].apply(lambda x : shuffle_list(123, x)).to_dict()
data['song_name_token'] = data['songs'].progress_apply(lambda songs : sum(preproc_shuffle_song(123, songs), []))
# data['song_name_token'] = data['songs'].progress_apply(lambda songs : sum([preprocess_string(song_meta.loc[song_id, 'song_name']) for song_id in random.sample(songs, len(songs))], []))
# album name도 추가하는 version
# data['song_name_token'] = data['songs'].progress_apply(lambda songs : sum([preproc_song(song_id) for song_id in songs], []))
song_name_dic = data['song_name_token'].to_dict()

100%|█████████████████████████████████| 115071/115071 [04:24<00:00, 434.64it/s]


- 문장 구성 : 노래 제목 + 태그 + 플레이리스트 제목

In [9]:
data = data.reset_index()
total = data.progress_apply(lambda x : song_name_dic[x['id']] + tag_dic[x['id']] + preprocess_string(x['plylst_title'], CUSTOM_FILTERS), axis = 1)

100%|████████████████████████████████| 115071/115071 [00:13<00:00, 8580.24it/s]


### Word2Vec Model Fitting

In [10]:
w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg)

### Calculate Embedding Vectors

In [11]:
ID = []   
vec = []
embedd = {}
for index, q in tqdm(pd.concat([train, val]).iterrows()):
    tmp_vec = 0
    for song_word in song_name_dic[q['id']]:
        try:
            tmp_vec += w2v_model.wv.get_vector(song_word) / len(song_name_dic[q['id']])
        except KeyError:
            pass
    for tag in tag_dic[q['id']]:
        try:
            tmp_vec += w2v_model.wv.get_vector(tag) / len(tag_dic[q['id']])
        except KeyError:
            pass
    for title_word in preprocess_string(q['plylst_title'], CUSTOM_FILTERS):
        try:
            tmp_vec += w2v_model.wv.get_vector(title_word) / len(preprocess_string(q['plylst_title'], CUSTOM_FILTERS))
        except KeyError:
            pass

    if type(tmp_vec) != int:
        embedd[str(q['id'])] = tmp_vec
        ID.append(str(q['id']))  
        vec.append(tmp_vec)

p2v_model.add(ID, vec)

115071it [09:38, 198.76it/s]


In [12]:
import pickle

with open("w2v_embed.txt", "wb") as file:
    pickle.dump(embedd, file)

### Calculate embedding vectors for Songs

In [ ]:
s2v_model = WordEmbeddingsKeyedVectors(size)

ID_song = []
vec_song = []
song_embed = {}
for index, q in tqdm(song_meta.iterrows()):
    tmp_vec = 0
    for word in preprocess_string(q['song_name'], CUSTOM_FILTERS):
        try:
            tmp_vec += w2v_model.wv.get_vector(word) / len(preprocess_string(q['song_name'], CUSTOM_FILTERS))
        except KeyError:
            pass
    if type(tmp_vec) != int:
        song_embed[q['song_name']] = tmp_vec
        ID_song.append(str(q['id']))
        vec_song.append(tmp_vec)
        
s2v_model.add(ID_song, vec_song)

### Make answers to submit

In [13]:
import dateutil.parser

# issue_date -> datetime으로 변환
def parse_issue_date(issue_date):
    issue_date = issue_date + int(issue_date % 10000 == 0) * 100 + int(issue_date % 100 == 0)
    return dateutil.parser.parse(str(issue_date))

In [14]:
most_results = pd.read_json('./arena_data/results/results.json', encoding='UTF-8')

In [15]:
answers = []
for index, q in tqdm(val.iterrows()):
    try:
        most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
        get_song = []
        get_tag = []
        for ID in most_id:
            get_song += song_dic[int(ID)]
            get_tag += tag_dic[int(ID)]
        '''
        get_song = list(pd.value_counts(get_song)[:200].index)
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        '''
        
        updt_date = dateutil.parser.parse(q['updt_date'])
        get_song = pd.Series(list(pd.value_counts(get_song)[:200].index))
        song_date = [parse_issue_date(song_meta.loc[song_id, 'issue_date']) for song_id in get_song]
        get_song_filtered = get_song[[x <= updt_date  for x in song_date]]
        
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        
        answers.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song_filtered)[:100],
            "tags": remove_seen(q["tags"], get_tag)[:10],
        })
    except:
        print('except : ', q)
        answers.append({
          "id": most_results.loc[index]["id"],
          "songs": most_results.loc[index]['songs'],
          "tags": most_results.loc[index]["tags"],
        }) 

# check and update answer
for n, q in enumerate(answers):
    if len(q['songs'])!=100:
        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]

291it [00:14, 25.03it/s]

except :  id                               141757
like_cnt                              0
plylst_title                     testtt
songs                                []
tags                                 []
updt_date       2018-07-03 18:27:03.000
Name: 286, dtype: object
except :  id                                31245
like_cnt                              6
plylst_title                     달리고싶을때
songs                                []
tags                                 []
updt_date       2016-09-10 17:56:49.000
Name: 289, dtype: object


377it [00:19, 23.18it/s]

except :  id                                22484
like_cnt                             20
plylst_title                       한의원~
songs                                []
tags                                 []
updt_date       2016-09-12 00:29:51.000
Name: 374, dtype: object


430it [00:21, 23.51it/s]

...

22882it [20:07, 24.29it/s]

except :  id                                31955
like_cnt                              6
plylst_title                   생생멜론리퀘스트
songs                                []
tags                                 []
updt_date       2006-12-21 07:05:02.000
Name: 22878, dtype: object


23009it [20:13, 23.50it/s]

except :  id                                 1550
like_cnt                              0
plylst_title                      루룰노래2
songs                                []
tags                                 []
updt_date       2018-07-13 16:15:15.000
Name: 23006, dtype: object


23015it [20:13, 18.97it/s]


In [16]:
write_json(answers, "results.json")

정답 확인해보기

In [26]:
print(val['plylst_title'][idx])
print([song_meta.loc[song_id, 'song_name'] for song_id in answers[idx]['songs']])

책읽을때 듣기좋은 뉴에이지 음악
['사랑하는 이들에게', '지금은 우리가 멀리 있을지라도', 'Kiss The Rain', 'Reminiscent (회상)', 'River Flows In You', 'Merry Christmas Mr. Lawrence', '따뜻한 그날', '오솔길', 'Blue Sea', '그대를 생각했던 순간들', 'May Be', 'Love Me', '마음정원', '얼음연못', '당신의 사랑이 늘 행복하기를', '화정학려 (華亭鶴&#21811;)', '그리워 한다는 것은', '환상(illusion fantasy)', 'Suddenly', 'Meditation', '창', '비가 그치고 난뒤(after the rain)', '그랬던 밤처럼', '그 집', '너의 순간들(Your Moments)', 'leaf', '공원에서', '사랑극', 'Last Carnival', '오늘은 왠지...', '참 좋은날 당신을 만났습니다', 'Happy Day', '여름의 조각들', '꽃이 되어 피다', '가장 조용했던 밤의 위로', '여행', '위로가 필요한 밤에', 'Daily Life', 'Lake Louise', 'This Morning', '둘이서 걷던 길', '난 당신을 원해요 (Je Te Veux / Erik Satie)', '이제는 안녕', '놀이터', '별의 바다 (Sea Of Stars)', 'Romance (Reminiscence)', '마음으로 가는 길', 'If I Love You.', '돌아갈 수 없는', '너의 별이 반짝일 때', '바람소리', 'Long Long Ago', '반갑다 봄아', 'The Way To School (학교가는 길)', '메이세컨', 'Merry Christmas Mr. Lawrence', '돌아오는 길 위에 서서', '그 시절, 너와 나', 'Fresh Morning', '바람따라 가는 기차여행', '프라하로 가는 야간열차', 'Happy And Sad', '동화 같은 이야기', '우리가 아름답던 날들',

In [18]:
[song_meta.loc[song_id, 'song_name'] for song_id in val.loc[44, 'songs']]

['뱅뱅뱅 (BANG BANG BANG)',
 'NO MERCY',
 '중독 (Overdose)',
 '에너제틱 (Energetic)',
 '말이야',
 'Man In Love (남자가 사랑할때)',
 'The Closer',
 '만만하니',
 '늑대와 미녀 (Wolf)',
 '기도 (I`ll Be Your Man)',
 '네게만 집착해',
 'Heartbeat',
 '기적 (ETERNITY)',
 'Mr. Simple',
 'G.O.O.D Luv',
 'CALL ME BABY',
 '딱 좋아 (Just right)',
 '로꾸거!!!',
 'A-CHA',
 'Hands Up',
 '태풍 (The Eye)',
 '다시 돌아와',
 'Lucky',
 '긴 생머리 그녀 (Miss Right)',
 'Fiction',
 'SOLO DAY',
 '하드캐리',
 '소방차 (Fire Truck)',
 'Ring Ding Dong',
 '좋겠어 (I wish)',
 'HER',
 '대박사건 (Crash)',
 'Lotto',
 '뛰뛰빵빵',
 'Something',
 '붐붐',
 '외톨이야',
 '거짓말',
 '쩔어',
 'Monster',
 'Lucifer',
 '닐리리맘보',
 '도원경 (桃源境)',
 '스모키걸 (Smoky Girl)',
 '대.다.나.다.너',
 '내꺼하자',
 '쏘리 쏘리 (Sorry, Sorry)',
 'Oh Little Girl',
 'To You',
 '이별공식',
 'Tell Me What To Do',
 '새들처럼',
 'Again＆Again',
 'SPY',
 'A',
 'Fantasy',
 'Shock',
 'Playdate',
 '바래',
 '호르몬 전쟁',
 'Y',
 '에라 모르겠다',
 '바람의 유령',
 'BAE BAE',
 'SUPER HERO',
 'Sherlock.셜록 (Clue+Note)',
 '산소 같은 너 (Love Like Oxygen)',
 '상남자 (Boy In Luv)',
 'Miracle',
 'Hey Ma

In [19]:
[song_meta.loc[song_id, 'song_name'] for song_id in answers[44]['songs']]

['Ice Cream Cake',
 '오늘부터 우리는 (Me Gustas Tu)',
 '음오아예 (Um Oh Ah Yeh)',
 'LOVE ME RIGHT',
 '심쿵해 (Heart Attack)',
 'CHEER UP',
 'Dumb Dumb',
 '러시안 룰렛 (Russian Roulette)',
 'OOH-AHH하게',
 '행복 (Happiness)',
 'SHAKE IT',
 'NoNoNo',
 'TT',
 'View',
 '첫 사랑니 (Rum Pum Pum Pum)',
 '위아래',
 'Mr. Chu (On Stage)',
 'I Got A Boy',
 'Touch My Body',
 '불타오르네 (FIRE)',
 '너 그리고 나 (NAVILLERA)',
 'Loving U (러빙유)',
 '유리구슬 (Glass Bead)',
 '링마벨 (Ring My Bell)',
 'FANTASTIC BABY',
 'JACKPOT',
 'Gee',
 'PARTY',
 '다른 남자 말고 너',
 '불장난',
 'Ah-Choo',
 'Remember',
 '빨간 맛 (Red Flavor)',
 '시간을 달려서 (Rough)',
 'Rookie',
 '너무너무너무',
 '이게 무슨 일이야',
 'Darling',
 '내가 제일 잘 나가',
 '사뿐사뿐',
 '넌 is 뭔들',
 'Decalcomanie (데칼코마니)',
 '아주 NICE',
 'Hot Summer',
 '만세',
 '마지막처럼',
 'Twinkle',
 'So Cool (쏘쿨)',
 'NU 예삐오 (NU ABO)',
 '기대해',
 'Lion Heart',
 'Bad Girl Good Girl',
 'Oh!',
 '빨개요',
 '아예 (Ah Yeah)',
 'Electric Shock',
 'KNOCK KNOCK',
 '얼굴 찌푸리지 말아요',
 '휘파람',
 '붐바야',
 'Dream Girl',
 '단발머리 (Short Hair)',
 'Very Good',
 '짧은 치마 (Miniskirt)',


## Class로 만든 버전

In [ ]:
# 목표 : 플레이리스트(노래, 태그, 플레이리스트 제목)이 주어지면, K개의 feature를 뽑도록 한다.
# 자연어 처리 모델을 구축
class PlaylistEmbedding:
    def __init__(self):
        # 파일에서 긁어온다.
        # 긁어올 파일 : train, val, song_meta
        self.train = pd.read_json('./data/train.json', encoding='UTF-8')
        self.val = pd.read_json('./data/val.json', encoding='UTF-8')
        self.song_meta = pd.read_json('./data/song_meta.json', encoding='UTF-8')
        self.most_results = pd.read_json('results.json', encoding='UTF-8')
        self.min_count = 10
        self.size = 100
        self.windows = 100
        self.sg = 1
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
    
    #플레이리스트 : (플레이리스트 제목과 모든 송의 제목을 gensim으로 preprocess, 장르도 우겨넣자) 
    def get_dic(self, train, val, song_meta):
        song_dic = {} # plylst id - song id
        song_name_dic = {} # plylst id - [preprocessed song name]
        tag_dic = {} # plylst id - tag
        data = pd.concat([train, val])
        data = data.set_index('id')
        
        song_dic = data['songs'].to_dict()
        tag_dic = data['tags'].to_dict()
        data['song_name_token'] = data['songs'].map(lambda x : sum(list(map(lambda xx : preprocess_string(song_meta.loc[xx, 'song_name']), x)), []))
        song_name_dic = data['song_name_token'].to_dict()
        
        '''
        for index, q in tqdm(data.iterrows()):
            song_name_dic[str(q['id'])] = sum(list(map(lambda x : preprocess_string(song_meta.loc[x]['song_name']), q['songs'])),[])
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        '''
        self.song_dic = song_dic
        self.song_name_dic = song_name_dic
        self.tag_dic = tag_dic
        
        data = data.reset_index()
        total = data.apply(lambda x : song_name_dic[x['id']] + tag_dic[x['id']] + preprocess_string(x['plylst_title']), axis = 1)
        # total = [x for x in total if len(x)>1]
        self.total = total
    
    # word2vec 모델
    def get_w2v(self):
        w2v_model = Word2Vec(self.total, min_count = self.min_count, size = self.size, window = self.windows, sg = self.sg)
        self.w2v_model = w2v_model
    
    # word2vec을 적합하고 embedded vector를 return
    def update_p2v(self, train, val,w2v_model):
        ID = []   
        vec = []
        embedd = {}
        for index, q in tqdm(pd.concat([train, val]).iterrows()):
            tmp_vec = 0
            for song_word in self.song_name_dic[str(q['id'])]:
                try:
                    tmp_vec += w2v_model.wv.get_vector(song_word)
                except KeyError:
                        pass
            for tag in self.tag_dic[str(q['id'])]:
                try:
                    tmp_vec += w2v_model.wv.get_vector(tag)
                except KeyError:
                        pass
            for title_word in preprocess_string(q['plylst_title']):
                #print(q['plylst_title'])
                try:
                    tmp_vec += w2v_model.wv.get_vector(title_word)
                except KeyError:
                        pass
                    
            if type(tmp_vec) != int:
                embedd[str(q['id'])] = tmp_vec
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
                
        self.embedd = embedd
        self.p2v_model.add(ID, vec)
        return embedd
    
    def get_results(self):
        answers = []
        for index, q in tqdm(self.val.iterrows()):
            try:
                most_id = [x[0] for x in self.p2v_model.most_similar(str(q['id']), topn=200)]
                get_song = []
                get_tag = []
                for ID in most_id:
                    get_song += self.song_dic[int(ID)]
                    get_tag += self.tag_dic[int(ID)]
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
            except:
                answers.append({
                  "id": self.most_results.loc[index]["id"],
                  "songs": self.most_results.loc[index]['songs'],
                  "tags": self.most_results.loc[index]["tags"],
                }) 
                
        # check and update answer
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results.loc[n]['songs'])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results.loc[n]['tags'])[:10-len(q['tags'])]  
        self.answers = answers

In [ ]:
playlist = PlaylistEmbedding()

In [ ]:
playlist.get_dic(playlist.train, playlist.val, playlist.song_meta)

In [ ]:
playlist.get_w2v()

In [ ]:
playlist.update_p2v(playlist.train, playlist.val, playlist.w2v_model)

In [ ]:
playlist.get_results()

In [ ]:
write_json(playlist.answers, "results.json")